In [1]:
# import all libraries etc.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import statsmodels.api as sm
import statsmodels.stats.api as sms
import scipy.stats as stats
%matplotlib inline
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
import warnings

In [2]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv("data/Xtrain.csv")
y = pd.read_csv('data/ytrain.csv')

test = pd.read_csv('data/Xtest.csv')

In [4]:
######for train dataset
sqft_basement_list= []
for num in data['sqft_basement'].values:
    if num == '?':
        sqft_basement_list.append(np.nan)
    else:
        sqft_basement_list.append(float(num))
data['sqft_basement']= sqft_basement_list
#cleaning year renovated        
data['yr_renovated'] = data['yr_renovated'].fillna(0).astype(int)
data['yr_renovated'] = np.where(data['yr_renovated'] == 0, data['yr_built'], data['yr_renovated'])

#waterfron cleaning
data['waterfront'].replace(np.nan, 0, inplace =True)

#view col cleaning
data['view'].replace(np.nan, 0 , inplace =True)

#basement
data['sqft_basement'].fillna(0, inplace= True)

#one house has a 33 bedroom house to 3:
data['bedrooms'].replace(33, 3, inplace=True)





######for test data
sqft_basement_list2= []
for num in test['sqft_basement'].values:
    if num == '?':
        sqft_basement_list2.append(np.nan)
    else:
        sqft_basement_list2.append(float(num))
test['sqft_basement']= sqft_basement_list2
#cleaning year renovated        
test['yr_renovated'] = test['yr_renovated'].fillna(0).astype(int)
test['yr_renovated'] = np.where(test['yr_renovated'] == 0, test['yr_built'], test['yr_renovated'])


#waterfron cleaning
test['waterfront'].replace(np.nan, 0, inplace =True)
#view col cleaning
test['view'].replace(np.nan, 0 , inplace =True)

#basement
test['sqft_basement'].fillna(0, inplace= True)

#one house has a 33 bedroom house to 3:
test['bedrooms'].replace(33, 3, inplace=True)


In [5]:
#########Dummying grade for train###########
grade_origin= data['grade']
df_dummy_grade= pd.get_dummies(grade_origin, drop_first= True) #avoid dummy trap
data = pd.concat([df_dummy_grade, data], axis=1)

#renaming the new Columns
#grade
mapper = {4:'Grade_4', 5: 'Grade_5', 6:'Grade_6',7:'Grade_7',
          8:'Grade_8',9:'Grade_9',10:'Grade_10', 11:'Grade_11',12:'Grade_12',13:'Grade_13'} 
data.rename(mapper=mapper, axis=1, inplace= True)




############Dummying grade for test###########
grade_origin2= test['grade']
df_dummy_grade2= pd.get_dummies(grade_origin2, drop_first= False) #avoid dummy trap
test = pd.concat([df_dummy_grade2, test], axis=1)

#renaming the new Columns
#grade
mapper2 = {4:'Grade_4', 5: 'Grade_5', 6:'Grade_6',7:'Grade_7',
          8:'Grade_8',9:'Grade_9',10:'Grade_10', 11:'Grade_11',12:'Grade_12',13:'Grade_13'} 
test.rename(mapper=mapper2, axis=1, inplace= True)

In [6]:
#making dummy for condition for train
condition_origin= data['condition']
df_cond_dum=pd.get_dummies(condition_origin, drop_first=True)
data= pd.concat([df_cond_dum, data], axis=1)

mapper3 = {1:'Condition_1',2:'Condition_2',3:'Condition_3',4:'Condition_4',5:'Condition_5'}
data.rename(mapper=mapper3, axis=1, inplace=True)



#dummy condition for test
condition_origin2= test['condition']
df_cond_dum2=pd.get_dummies(condition_origin2, drop_first=True)
test= pd.concat([df_cond_dum2, test], axis=1)

mapper3 = {1:'Condition_1',2:'Condition_2',3:'Condition_3',4:'Condition_4',5:'Condition_5'}
test.rename(mapper=mapper3, axis=1, inplace=True)

In [7]:
# adding some interactions
data['age'] = 2021 - data['yr_built']

#decided to combine Bath and Bedroom into a ratio (Bed:Bath) since its a commonly used metric fpr real estate

data['bed:bath'] = data['bedrooms']/data['bathrooms']

data = data.loc[:,~data.columns.duplicated()]

In [8]:
#Dummying the zip train
zip_origin= data['zipcode']
df_zip= pd.get_dummies(zip_origin, drop_first= True)
data = pd.concat([df_zip, data], axis=1)

data = data.drop('zipcode',axis = 1)


#dummy the zip test
zip_origin2= test['zipcode']
df_zip2= pd.get_dummies(zip_origin2, drop_first= True)
test = pd.concat([df_zip2, test], axis=1)

test = test.drop('zipcode',axis = 1)



In [9]:
# adding some interactions train
data['age'] = 2021 - data['yr_built']

#decided to combine Bath and Bedroom into a ratio (Bed:Bath) since its a commonly used metric fpr real estate

data['bed:bath'] = data['bedrooms']/data['bathrooms']

data = data.loc[:,~data.columns.duplicated()]




# adding some interactions test
test['age'] = 2021 - test['yr_built']

#decided to combine Bath and Bedroom into a ratio (Bed:Bath) since its a commonly used metric fpr real estate

test['bed:bath'] = test['bedrooms']/test['bathrooms']

test = test.loc[:,~test.columns.duplicated()]

In [10]:
#drop test
data.drop(['view', 'grade', 'bedrooms', 'bathrooms', 'condition', 'yr_renovated'], axis=1, inplace=True)

#drop train
test.drop(['view', 'grade', 'bedrooms', 'bathrooms', 'condition', 'yr_renovated'], axis=1, inplace=True)

In [11]:

#drop test
data = data.drop(['Grade_13', 'sqft_above', "yr_built",'date'], axis=1)

#drop train
test = test.drop(['Grade_13', 'sqft_above', "yr_built",'date'], axis=1)


In [12]:
x_train = data
y

,price
0,529000.00
1,253000.00
2,745000.00
3,545000.00
4,390000.00
...,...
16192,440000.00
16193,572000.00
16194,299800.00
16195,245000.00


In [13]:
lr = LinearRegression()

In [14]:
scores_simple = cross_validate(
                    lr, x_train, y, cv=5, 
                    return_train_score=True)

In [15]:
lr.fit(x_train, y)
print(lr.score(x_train, y))

0.8261355890673305


In [16]:
y_hat_train = lr.predict(x_train)

In [17]:
train_mse = mean_squared_error(y, y_hat_train, squared =False)
print('Train Mean Error:', train_mse)

Train Mean Error: 152750.24369515275


In [18]:
y_hat_test = lr.predict(test)

In [19]:
y_hat_test

array([[ 161969.66073395],
       [ 540938.07934476],
       [ 340080.70554999],
       ...,
       [1332158.36617207],
       [1349228.88117586],
       [ 178439.84196886]])

In [20]:
y_test_fake = np.full((5400,1), 0)

In [21]:
from sklearn.metrics import r2_score

# fake predictions using the mean of y_train.
y_hat_test = np.full((5400,1), np.mean(y))

r2_score(y_hat_test, y_test_fake)

0.0

In [22]:
len(y_hat_test)

5400

In [23]:
np.savetxt('Group2_bakeoff.csv', y_hat_test, delimiter=',',header = 'price_estimate')

In [24]:
ls

Bakeoff.ipynb        README.md            student.ipynb
Bakeoff_final.ipynb  Untitled.ipynb       test-el.ipynb
Group2_bakeoff.csv   data/
PROJ2_updated.ipynb  picture/


In [25]:
y_hat_test.shape

(5400, 1)

In [26]:
df = pd.read_csv("Group2_bakeoff.csv")

In [27]:
df

,# price_estimate
0,541284.46
1,541284.46
2,541284.46
3,541284.46
4,541284.46
...,...
5395,541284.46
5396,541284.46
5397,541284.46
5398,541284.46
